
The objective of this script was to extract a comprehensive list of USA Olympians fromn the Team USA website. The process involved several challenges:

1. Pagination and Ads: Initially, the table displayed only 10 Olympians along with an ad and a "View More Athletes" button. Clicking this button revealed 10 additional athletes along with another ad and button. With a over 590 Olympians to retrieve, the script needed to repeatedly click the "View More Athletes" button to load all entries.

2. Pop up Interference: a pop-up ad obscured the "View More Athletes" button, requiring the script to handle this distraction.

3. Table Structure: Each row in the Olympian table contained a nested header with additional information. The script had to extract the Olympian names from these nested headers and then combine this information with the rest of the data for each olympian.

Overall, the script efficiently navigates through dynamic content and handles complex table structures to compile a complete list of athletes.

In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Configure Chrome Options
chrome_options = Options()
chrome_options.add_argument("--headless=new") # Headless allows users to run automated script without the borwser window visible

# Initialize WebDriver
driver = webdriver.Chrome(options=chrome_options)
url = r"https://www.teamusa.com/paris-2024/olympics#Team_USA_Qualified_Athletes_csdd"
driver.get(url)

Ad
Below code should be ammended to wait until the button appears and put into a try block

In [3]:
close_ad_button = driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/div/div[2]/button")
ActionChains(driver).scroll_to_element(close_ad_button).click(close_ad_button).perform()
driver.implicitly_wait(5)

In [4]:
# Continuously click 'View More Athletes' button 
while True:
    try:
        button = driver.find_element(By.XPATH, "/html/body/div[1]/main/div[8]/div/div[4]/button")

        ActionChains(driver).scroll_to_element(button).click(button).perform()

        driver.implicitly_wait(5)
    except (NoSuchElementException, ElementNotInteractableException):
        print("Button is no longer available")
        break

Button is no longer available


In [6]:
# Parse page with BS
src = driver.page_source
driver.quit()
parser = BeautifulSoup(src, "lxml")

# Find table
table = parser.find("table", attrs = {"class": "chakra-table css-1o7rs0n"})
if not table:
    raise Exception("Table not found")

headers = [th.text.strip() for th in table.findAll('th')][:4] # Pull text values of headers

rows = table.findAll('tr')[1:]


In [7]:
names = [p.getText().strip() for row in rows for p in row.findAll("p", attrs={"class": "chakra-text css-mpfnrl"})]
olympian_info = [
        [td.getText().strip() for td in row.findAll('td') if "css-egy9nu" not in td.get('class', [])]
        for row in rows
    ]
olympian_info  = [info for info in olympian_info if info]
combined_list = [[names[i]] + olympian_info[i] for i in range(len(names))]
o_data = pd.DataFrame(combined_list, columns=headers)

In [8]:
o_data

,Name,Sport,Education,Hometown
0,Paxten Aaronson,Soccer,,"Medford, NJ"
1,Katelyn Abeln,Shooting,Ohio State University,"Douglasville, GA"
2,Bam Adebayo,Basketball,University of Kentucky,"Newark, NJ"
3,Nia Akins,Track and Field,University of Pennslyvania,"San Diego, CA"
4,Korbin Albert,Soccer,University of Notre Dame,"Grayslake, IL"
...,...,...,...,...
592,Rose Zhang,Golf,"Pacific Academy (Irvine, Calif.) ’21, Stanford...","Irvine, CA"
593,Beiwen Zhang,Badminton,,"Las Vegas, NV"
594,Zach Ziemek,Track and Field,"University of Wisconsin, Madison '16, Sociology","Itasca, IL"
595,Maddie Zimmer,Field Hockey,Hershey Northwestern '24,"Hershey, PA"


In [12]:
pd.DataFrame.to_excel(o_data, "T:\\AArete Client Work\\CODE\\TeamMember\\sclark\\Python\\Webscrape\\olympians.xlsx")